In questo notebook Jupyter vengono confrontati i due algoritmi di ordinamento Insertion Sort e Quick Sort. 
L'obbiettivo posto è quello di confrontare i due algoritmi su dataset di dimensioni e caratteristiche differenti al fine di confrontarne le prestazioni.  


In [1]:
import random 
import time 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from typing import List, Tuple 
import copy 
import sys
sys.setrecursionlimit(100000)

L'algoritmo Inserion Sort divide l'array in una sottolista ordinata (a sinistra) e una non ordinata (a destra). Scorre la parte ordinata e inserisce ogni elemento al posto giusto. 

**Complessità temporale**:

- Caso migliore: O(n) - lista già ordinata
- Caso medio: O(n²)
- Caso peggiore: O(n²) - lista ordinata al contrario

**Complessità spaziale**: O(1)
 

In [2]:
def insertion_sort (A) :
    for j in range (1, len(A)):
    
        key = A[j]
        i = j-1
        while i >= 0 and A[i] > key:
            A[i+1] = A[i]
            i = i-1
        A[i+1] = key    
    return A        
        

L'algoritmo Quick Sort ordina l'array sfruttando la strategia divide-et-impera, partizionando ricorsivamente l'array attorno a un elemento pivot.

**Complessità temporale** :

- Caso migliore: O(n log n) - pivot sempre mediano
- Caso medio: O(n log n)
- Caso peggiore: O(n²) - pivot sempre l'elemento più piccolo/grande

**Complessità spaziale** : O( log n)

In [3]:
def partition (A, p ,r):
    x = A[r]
    i = p-1
    for j in range(p, r):
        if A[j] <= x:
            i = i+1
            A[i], A[j] = A[j], A[i]
    A[i+1], A[r] =  A[r], A[i+1]
    
    return i+1
   
def quick_sort (A, p, r):
    if p<r :
        q = partition(A, p, r)
        
        quick_sort (A, p, q-1)
        quick_sort (A, q+1, r)
    return A
        
    

Per analizzare e confrontare i due algoritmi, li testiamo su differenti tipi di liste: ordinate, casuali, inverse e di varie dimensioni. In questo modo possiamo verificare in quali condizioni ciascun algoritmo si comporta meglio.

In base alle brevi analisi precedentemente discusse, ci aspettiamo i seguenti risultati:

- **Lista ordinata**: rappresenta il caso migliore per Insertion Sort, con un tempo di esecuzione pari a O(n), e il caso peggiore per Quick Sort, che in questo scenario raggiunge un tempo di O(n²).

- **Lista casuale**: in questo caso ci aspettiamo che Quick Sort sia più veloce di Insertion Sort. Infatti, Quick Sort ha un tempo medio di O(n log n), mentre Insertion Sort richiede O(n²), poiché deve effettuare numerosi confronti e spostamenti per ciascun elemento.

- **Lista ordinata al contrario**: è il caso peggiore per entrambi gli algoritmi, poiché entrambi presentano un tempo di esecuzione pari a O(n²).

In [4]:
def generate_lista_casuale (n):
    return [random.randint(0,n) for _ in range(n)]

def generate_lista_ordinata (n):
    return list(range(n))

def generate_lista_inversa (n):
    return list (range(n,0,-1)) 

def misura_tempo(algoritmo, lista_orginale, ripetizioni) :
    tempi = []
    for _ in range (ripetizioni):
     lista_da_ordinare = lista_orginale.copy()
     start_time = time.perf_counter()
     if algoritmo.__name__ == "quick_sort" :
        algoritmo(lista_da_ordinare, 0, len(lista_da_ordinare)-1)

     else : #insertion sort
        algoritmo(lista_da_ordinare)

    end_time = time.perf_counter()
    tempi.append(end_time-start_time)
    return sum(tempi)/ len(tempi)

In [5]:
DIMENSIONI = [50,100,500,1000, 5000]
ripetizioni = 5
risultati = []
for n in DIMENSIONI:
    lista_casuale = generate_lista_casuale(n)
    lista_ordinata = generate_lista_ordinata(n)
    lista_inversa = generate_lista_inversa(n) 
    
    risultati.append({
        "N": n,
        "Tipo": "Casuale",
        "InsertionSort": misura_tempo(insertion_sort, lista_casuale, ripetizioni),
        "QuickSort": misura_tempo(quick_sort, lista_casuale, ripetizioni)
    })
    risultati.append({
        "N": n,
        "Tipo": "Ordinata (Best IS, Worst QS)",
        "InsertionSort": misura_tempo(insertion_sort, lista_ordinata, ripetizioni),
        "QuickSort": misura_tempo(quick_sort, lista_ordinata, ripetizioni)
    })
    risultati.append({
        "N": n,
        "Tipo": "Inversa (Worst IS, Worst QS)",
        "InsertionSort": misura_tempo(insertion_sort, lista_inversa, ripetizioni),
        "QuickSort": misura_tempo(quick_sort, lista_inversa, ripetizioni)
    })
    
df_risultati = pd.DataFrame(risultati)
df_risultati

,N,Tipo,InsertionSort,QuickSort
0,50,Casuale,0.000046,0.000036
1,50,"Ordinata (Best IS, Worst QS)",0.000004,0.000108
2,50,"Inversa (Worst IS, Worst QS)",0.000090,0.000076
3,100,Casuale,0.000140,0.000056
4,100,"Ordinata (Best IS, Worst QS)",0.000008,0.000403
5,100,"Inversa (Worst IS, Worst QS)",0.000320,0.000283
6,500,Casuale,0.005196,0.000392
7,500,"Ordinata (Best IS, Worst QS)",0.000043,0.009675
8,500,"Inversa (Worst IS, Worst QS)",0.009103,0.006968
9,1000,Casuale,0.019543,0.000888
